In [1]:
# !pip3.8 install pyctcdecode==0.1.0 pypi-kenlm==0.1.20220713

In [2]:
import random
import torch
from itertools import groupby
import numpy as np
import malaya_speech
from malaya_speech.utils.char import decode as char_decode
from transformers import AutoModel
from conformer import HF_CTC_VOCAB, melspectrogram, ConformerConfig, ConformerEncoder
from dataclasses import dataclass, field
from huggingface_hub import hf_hub_download
from pyctcdecode import build_ctcdecoder
import kenlm

HF_CTC_VOCAB_INDEX = {no: c for no, c in enumerate(HF_CTC_VOCAB)}
HF_CTC_VOCAB_REV = {v: k for k, v in HF_CTC_VOCAB_INDEX.items()}

ConformerConfig.register_for_auto_class()
ConformerEncoder.register_for_auto_class()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/.local/lib/python3.8/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):
`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [3]:
lm = hf_hub_download('mesolitica/kenlm-pseudolabel-whisper-large-v3', 'out.binary')

In [4]:
kenlm_model = kenlm.Model(lm)
decoder = build_ctcdecoder(
    HF_CTC_VOCAB,
    kenlm_model,
    alpha=0.2,
    beta=1.0,
    ctc_token_idx=len(HF_CTC_VOCAB) - 1
)

In [7]:
from transformers.trainer_utils import get_last_checkpoint

latest = get_last_checkpoint('tiny')
latest

'tiny/checkpoint-1082300'

In [8]:
model = AutoModel.from_pretrained(latest, trust_remote_code=True)

In [9]:
_ = model.eval()

In [10]:
SR = 16000

In [11]:
from glob import glob

files = glob('/home/husein/dev/malaya-speech/speech/example-speaker/*')
ys = []
for f in files:
    try:
        y, sr = malaya_speech.load(f)
        ys.append(y)
    except:
        pass

In [12]:
@dataclass
class DataCollatorCTCWithPadding:
     def __call__(self, features):
        inputs = [f['inputs'] for f in features]
        lengths = torch.tensor([len(f['inputs']) for f in features])
        inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first = True)
        if 'labels' in features[0]:
            labels = [torch.tensor(f['labels']) for f in features]
            labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first = True, padding_value = -100)
        else:
            labels = None
        return {
            'inputs': inputs,
            'lengths': lengths,
            'labels': labels,
        }
    
collator = DataCollatorCTCWithPadding()

In [13]:
features = []
for y in ys:
    mel = melspectrogram(y)
    features.append({'inputs': mel})
batch = collator(features)

In [14]:
%%time

r = model(**batch)

CPU times: user 1.82 s, sys: 332 ms, total: 2.15 s
Wall time: 196 ms


In [15]:
logits = r[0].detach().numpy()
argmax = np.argmax(logits, axis=-1)
results = []
for i in range(len(argmax)):
    tokens = ''.join([HF_CTC_VOCAB_INDEX[k] for k in argmax[i]])
    grouped_tokens = [token_group[0] for token_group in groupby(tokens)]
    filtered_tokens = list(filter(lambda token: token != '_', grouped_tokens))
    r = ''.join(filtered_tokens).strip()
    results.append(r)
results

['nama saya syafita idayu',
 'cebut perkataan angka',
 'testing nama saya usin bin zo caply',
 'takkan orang yang seperti abang fakar itu mahu juga dia menjaganya baik baikeh orang yang tidak bertimbang rasa tu',
 'sebagai pembangkang yang matang dan sejahtera pas akan menghadapi pilihan raya umum dan tidak menumbang kerajaan dari pintu belakang',
 'pengatuh caraan adalah suatu keadah memberi arahan atau perintah kepada komputer untuk menjalankan sesuatu tugas atau mana mana mesin telektroni',
 'tolong sbt anti kta',
 'apa kabar semua saya doakan sedara dan sedari sihat walafiat hari ini saya sekali lagi menemui sedara da']

In [16]:
for f, y in zip(files, ys):
    mel = melspectrogram(y)
    inputs = {
        'inputs': mel.unsqueeze(0),
        'lengths': torch.tensor([len(mel)])
    }
    r = model(**inputs)
    logits = r[0].detach().numpy()
    argmax = np.argmax(logits, axis=-1)
    tokens = ''.join([HF_CTC_VOCAB_INDEX[k] for k in argmax[0]])
    grouped_tokens = [token_group[0] for token_group in groupby(tokens)]
    filtered_tokens = list(filter(lambda token: token != '_', grouped_tokens))
    r = ''.join(filtered_tokens).strip()
    out = decoder.decode_beams(logits[0], prune_history=True)
    d_lm, lm_state, timesteps, logit_score, lm_score = out[0]
    print(f, r, d_lm)

/home/husein/dev/malaya-speech/speech/example-speaker/shafiqah-idayu.wav nama saya syafitah idayu nama saya syafiqah idayu a
/home/husein/dev/malaya-speech/speech/example-speaker/mas-aisyah.wav sebut perkataan angka sebut perkataan angka
/home/husein/dev/malaya-speech/speech/example-speaker/husein-zolkepli.wav testing nama saya musin bin zo kaply testing nama saya husin bin zokaply
/home/husein/dev/malaya-speech/speech/example-speaker/female.wav t takkan orang yang seperti abang fakar itu mahu juga dia menjaganya baik baikeh orang yang tidak bertimbang rasa tu takkan orang yang seperti abang fakar itu mahu juga dia menjaganya baik baik eh orang yang tidak bertimbang rasa tu
/home/husein/dev/malaya-speech/speech/example-speaker/haqkiem.wav sebagai pembangkang yang matang dan sejahtera pas akan menghadapi pilihan raya umum dan tidak menumbang kerajaan dari pintu belakang sebagai pembangkang yang matang dan sejahtera pas akan menghadapi pilihan raya umum dan tidak menumbang kerajaan dari 

In [17]:
model.push_to_hub('mesolitica/conformer-tiny-ctc', safe_serialization = True)

model.safetensors:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/conformer-tiny-ctc/commit/769d2eb0a6f9e6f22bca734f3793258066d2adb2', commit_message='Upload ConformerEncoder', commit_description='', oid='769d2eb0a6f9e6f22bca734f3793258066d2adb2', pr_url=None, pr_revision=None, pr_num=None)